# GAM requirements in iJL1678b-ME and iEco1689-ME

Code for reproducing panels Figure 2d and Supplementary Fig. 2

## Setup environment

In [ ]:
import coralme
from coralme.builder.helper_functions import *
import pickle
import pandas
import os
import tqdm
import json
import matplotlib.pyplot as plt
import numpy as np
import sympy
pandas.set_option('display.max_colwidth', None)
plt.rcParams['svg.fonttype'] = 'none'

In [ ]:
me = coralme.io.json.load_json_me_model('published/ecoli/MEModel-step2-iJL1678b-ME.json')
me.add_boundary(me.metabolites.li_c, type = 'sink')
me.optimize(max_mu = 1.0, verbose = False)

In [ ]:
# construct the LP problem to accelerate feasibility checks
lp = me.construct_lp_problem(lambdify = True)

In [ ]:
# 30min, 15 CPUs
def get_feas(mu):
    me.feasibility({sympy.Symbol('mu', positive = True) : mu}, **{'lp' : lp})
    return me.solution

opt = me.solution.objective_value
mus = np.arange(0.01, opt-0.1, 0.01)

import multiprocessing
with multiprocessing.Pool(multiprocessing.cpu_count()-1) as pool:
    fluxes = pool.map(get_feas, mus)

In [ ]:
types = set()
for rxn in me.reactions:
    types.add(type(rxn))
types = sorted(types, key = lambda x: str(x))
# types

In [ ]:
import coralme, sympy

directly_modeled = {}

def get_reactions(met, rxn_type):
    value = sympy.Float(0)
    reactions = []
    
    for rxn in met.reactions:
        if isinstance(rxn, (rxn_type)) and rxn.metabolites[met] < 0:
            reactions.append(rxn.id)
            value += rxn.metabolites[met] * fluxes[idx][rxn.id]
    
    return value, reactions

for rxn_type in types:
    directly_modeled[rxn_type] = {}
    for met in [ 'atp_c', 'utp_c', 'gtp_c', 'ctp_c', 'datp_c', 'dttp_c', 'dgtp_c', 'dctp_c', ]:
        directly_modeled[rxn_type][met] = []
        for idx, mu in enumerate(mus):
            value, reactions = get_reactions(me.metabolites.get_by_id(met), rxn_type)
            value = value.subs({ me.mu : me.solution.objective_value })
            directly_modeled[rxn_type][met].append(value)
            
df_directly_modeled = pandas.DataFrame.from_dict(directly_modeled)

In [ ]:
from scipy.stats import linregress

filter_out = [ 
    coralme.core.reaction.MEReaction, 
    coralme.core.reaction.MetabolicReaction,
    coralme.core.reaction.GenericFormationReaction
]

slopes = df_directly_modeled.applymap(lambda x: linregress(mus, [ -1. * float(x) for x in x])[0])
slopes = slopes.loc[:, ~slopes.columns.isin(filter_out)]

slopes.columns = [
    'Complex Formation', 
    'PostTranslation', 
    'Summary Variables', 
    'Transcription', 
    'Translation', 
    'tRNA Charging'
]

slopes.index = ['ATP', 'UTP', 'GTP', 'CTP', 'dATP', 'dTTP', 'dGTP', 'dCTP']
slopes = slopes.replace(0, np.nan)

In [ ]:
import seaborn

fig, ax = plt.subplots(1, 1, figsize = (4*1, 3*1.5), dpi = 100)

cg = seaborn.heatmap(
    slopes.T[['ATP', 'UTP', 'GTP', 'CTP']], 
    annot = True, cmap = 'Blues', vmax = 1., ax = ax,
    cbar_kws = {'label': 'Flux ($mmol / gDCW / h$)'})

plt.savefig('GAM_calculation.png', format = 'png', dpi = 300, 
            edgecolor = 'w', facecolor = 'w', bbox_inches = 'tight')
plt.savefig('GAM_calculation.svg', format = 'svg', dpi = 300, 
            edgecolor = 'w', facecolor = 'w', bbox_inches = 'tight')
plt.show()

In [ ]:
np.round(slopes.loc['ATP', 'Translation'], 2), np.round(slopes.loc['GTP', 'Translation'], 2)